In [258]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
import pyarrow as pa

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

In [3]:
SRC_DIR = './data-normalized'
LG_SS_HTML_DICT_STR = load_file(f'{SRC_DIR}/league_seasons_html.txt')
LG_SS_HTML_DICT = ast.literal_eval(LG_SS_HTML_DICT_STR)

LG_SS_LIST = list(LG_SS_HTML_DICT.keys())
LG_SS_HTML = LG_SS_LIST[1]
LG_SS_DIR = parse_league_id(LG_SS_HTML)['body']
LG_SS_DIR

'/leagues/NBA_2023'

In [ ]:
GL_TYPES = ['tgl_advanced','tgl_basic']
AG_TYPES = ['stats_expd_avg','stats_roll_05_avg','stats_roll_10_avg','stats_roll_15_avg','stats_roll_20_avg']
NM_TYPES = ['raw','norm_minmax','norm_ranking','norm_robust','norm_standard']
GLAN_TYPES_LIST = [f'{tgl_type}/{agg_type}/{norm_type}' for tgl_type,agg_type,norm_type in itertools.product(GL_TYPES,AG_TYPES,NM_TYPES)]
# TGL_AGG_NORM_TYPES_LIST

In [ ]:
GLAN_TYPE = 'tgl_basic/stats_expd_avg/norm_standard' 
SRC_GLAN_DIR = f'{SRC_DIR}/{LG_SS_DIR}/{GLAN_TYPE}'

# Load all data from the folder
GLAN_PER_RD_CSV_LIST = get_all_files(f'{SRC_GLAN_DIR}')
GLAN_PER_RD_DF_LIST = []
for PER_RD_CSV in GLAN_PER_RD_CSV_LIST:
    GLAN_PRD_DF = pd.read_csv(f'{SRC_GLAN_DIR}/{PER_RD_CSV}',header=[0,1],index_col=[0,1])
    GLAN_PER_RD_DF_LIST.append(GLAN_PRD_DF)

GLAN_SS_PER_RD_DF = pd.concat(GLAN_PER_RD_DF_LIST).sort_index(axis=0)
GLAN_SS_PER_RD_DF.index.names = ['index','Team_id']
print(GLAN_SS_PER_RD_DF.info())
GLAN_SS_PER_RD_DF.head(10)

In [145]:
SRC_DIR = './data-parsed'
TGL_BASIC_INFO_DF_LIST = []
for TM_SS_HTML in LG_SS_HTML_DICT[LG_SS_HTML]:
    TM_SS_DIR = parse_team_season_id(TM_SS_HTML)['body'] # teams/BOS/2023
    TGL_BASIC_DF = pd.read_csv(f'{SRC_DIR}/{TM_SS_DIR}/tgl_basic.csv',header=[0,1]).rename(columns={'Unnamed: 0_level_0':'Match','Unnamed: 0_level_1':'index'})['Match']
    TGL_BASIC_DF = TGL_BASIC_DF.set_index('Boxscores_html_id').rename(columns={'Tm_html_id':'Team_id'})[['index','Team_id','H/A']]
    TGL_BASIC_INFO_DF_LIST.append(TGL_BASIC_DF)

TM_IDX_FACTS_DF = pd.concat(TGL_BASIC_INFO_DF_LIST).sort_index(axis=0)
TM_IDX_FACTS_DF

TGL_BASIC_INFO_DF_LIST = []
for BS_INDEX in sorted(pd.unique(TM_IDX_FACTS_DF.index)):
    BS_TM_DF = TM_IDX_FACTS_DF.loc[BS_INDEX]
    TGL_BASIC_INFO_DF_LIST.append(pd.concat([BS_TM_DF,BS_TM_DF.iloc[::-1]],axis=1,keys=['Team','Opp']))
    
TM_OPP_IDX_FACTS_DF = pd.concat(TGL_BASIC_INFO_DF_LIST)
TM_OPP_IDX_FACTS_DF.set_index(pd.MultiIndex.from_frame(TM_OPP_IDX_FACTS_DF['Team'][['index','Team_id','H/A']]),inplace=True,drop=False)
DUMMY_IDX = pd.DataFrame([[-1]*6],columns=TM_OPP_IDX_FACTS_DF.columns,index=pd.MultiIndex.from_tuples([(-1,-1,-1)],names=['index','Team_id','H/A']))
TM_OPP_IDX_FACTS_DF = pd.concat([DUMMY_IDX,TM_OPP_IDX_FACTS_DF]).sort_index(axis=0)
TM_IDX_FACTS_DF = TM_OPP_IDX_FACTS_DF.loc[:,('Team',slice(None))]
OPP_IDX_FACTS_DF = TM_OPP_IDX_FACTS_DF.loc[:,('Opp',slice(None))]

# TM_OPP_IDX_FACTS_DF = TM_OPP_IDX_FACTS_VEN_DF.droplevel(2,axis=0).drop(columns=['H/A'],level=1)
# TM_IDX_FACTS_DF = TM_OPP_IDX_FACTS_DF.loc[:,('Team',slice(None))]
# OPP_IDX_FACTS_DF = TM_OPP_IDX_FACTS_DF.loc[:,('Opp',slice(None))]
TM_OPP_IDX_FACTS_DF
# TM_OPP_IDX_FACTS_DF

Team                             Opp   
                               index               Team_id H/A index   
index Team_id              H/A                                         
-1    -1                   -1     -1                    -1  -1    -1  \
 0    /teams/ATL/2023.html  1      0  /teams/ATL/2023.html   1     0   
      /teams/BOS/2023.html  1      0  /teams/BOS/2023.html   1     0   
      /teams/BRK/2023.html  1      0  /teams/BRK/2023.html   1     0   
      /teams/CHI/2023.html  0      0  /teams/CHI/2023.html   0     0   
      /teams/CHO/2023.html  0      0  /teams/CHO/2023.html   0     0   
      /teams/CLE/2023.html  0      0  /teams/CLE/2023.html   0     0   
      /teams/DAL/2023.html  0      0  /teams/DAL/2023.html   0     0   
      /teams/DEN/2023.html  0      0  /teams/DEN/2023.html   0     0   
      /teams/DET/2023.html  1      0  /teams/DET/2023.html   1     0   
      /teams/GSW/2023.html  1      0  /teams/GSW/2023.html   1     0   
      /teams/HOU/2023.html  0      0  /teams/HOU/2023.html   0     0   
      /teams/IND/2023.html  1      0  /teams/IND/2023.html   1     0   
      /teams/LAC/2023.html  0      0  /teams/LAC/2023.html   0     1   
      /teams/LAL/2023.html  0      0  /teams/LAL/2023.html   0     0   
...                              ...                   ...  ..   ...   
 81   /teams/MIA/2023.html  1     81  /teams/MIA/2023.html   1    81   
      /teams/MIL/2023.html  0     81  /teams/MIL/2023.html   0    81   
      /teams/MIN/2023.html  1     81  /teams/MIN/2023.html   1    81   
      /teams/NOP/2023.html  0     81  /teams/NOP/2023.html   0    81   
      /teams/NYK/2023.html  1     81  /teams/NYK/2023.html   1    81   
      /teams/OKC/2023.html  1     81  /teams/OKC/2023.html   1    81   
      /teams/ORL/2023.html  0     81  /teams/ORL/2023.html   0    81   
      /teams/PHI/2023.html  0     81  /teams/PHI/2023.html   0    81   
      /teams/PHO/2023.html  1     81  /teams/PHO/2023.html   1    81   
      /teams/POR/2023.html  1     81  /teams/POR/2023.html   1    81   
      /teams/SAC/2023.html  0     81  /teams/SAC/2023.html   0    81   
      /teams/SAS/2023.html  0     81  /teams/SAS/2023.html   0    81   
      /teams/TOR/2023.html  1     81  /teams/TOR/2023.html   1    81   
      /teams/UTA/2023.html  0     81  /teams/UTA/2023.html   0    81   
      /teams/WAS/2023.html  1     81  /teams/WAS/2023.html   1    81   

                                                          
                                             Team_id H/A  
index Team_id              H/A                            
-1    -1                   -1                     -1  -1  
 0    /teams/ATL/2023.html  1   /teams/HOU/2023.html   0  
      /teams/BOS/2023.html  1   /teams/PHI/2023.html   0  
      /teams/BRK/2023.html  1   /teams/NOP/2023.html   0  
      /teams/CHI/2023.html  0   /teams/MIA/2023.html   1  
      /teams/CHO/2023.html  0   /teams/SAS/2023.html   1  
      /teams/CLE/2023.html  0   /teams/TOR/2023.html   1  
      /teams/DAL/2023.html  0   /teams/PHO/2023.html   1  
      /teams/DEN/2023.html  0   /teams/UTA/2023.html   1  
      /teams/DET/2023.html  1   /teams/ORL/2023.html   0  
      /teams/GSW/2023.html  1   /teams/LAL/2023.html   0  
      /teams/HOU/2023.html  0   /teams/ATL/2023.html   1  
      /teams/IND/2023.html  1   /teams/WAS/2023.html   0  
      /teams/LAC/2023.html  0   /teams/LAL/2023.html   1  
      /teams/LAL/2023.html  0   /teams/GSW/2023.html   1  
...                                              ...  ..  
 81   /teams/MIA/2023.html  1   /teams/ORL/2023.html   0  
      /teams/MIL/2023.html  0   /teams/TOR/2023.html   1  
      /teams/MIN/2023.html  1   /teams/NOP/2023.html   0  
      /teams/NOP/2023.html  0   /teams/MIN/2023.html   1  
      /teams/NYK/2023.html  1   /teams/IND/2023.html   0  
      /teams/OKC/2023.html  1   /teams/MEM/2023.html   0  
      /teams/ORL/2023.html  0   /teams/MIA/2023.html   1  
      /teams/PHI/2023.html  0   /teams/BRK/2023.h

Get previous games

In [146]:
# q: How to query using the second level of the index?
# a: Use the .loc accessor
TM_IDX_SHFTD_DF = []
for team in pd.unique(TM_IDX_FACTS_DF[('Team','Team_id')]):
    TM_IDX_FACTS_PER_TM_DF = TM_IDX_FACTS_DF.loc[(slice(None),team),:]
    TM_IDX_SHFTD_PER_TM_DF_LIST = [TM_IDX_FACTS_PER_TM_DF.rename(columns={'Team':f'Team_Curr_Gm'})]
    for i in range(1,11):
        TM_IDX_SHFTD_PER_TM_DF_I = TM_IDX_FACTS_PER_TM_DF.copy().shift(i,fill_value=-1) 
        TM_IDX_SHFTD_PER_TM_DF_I = TM_IDX_SHFTD_PER_TM_DF_I
        Gm_No = f'0{i}' if i < 10 else f'{i}'
        TM_IDX_SHFTD_PER_TM_DF_I.rename(columns={'Team':f'Team_Prev_Gm_{Gm_No}'},inplace=True)
        TM_IDX_SHFTD_PER_TM_DF_LIST.append(TM_IDX_SHFTD_PER_TM_DF_I)
    TM_IDX_SHFTD_DF_PER_TM = pd.concat(TM_IDX_SHFTD_PER_TM_DF_LIST,axis=1)
    TM_IDX_SHFTD_DF.append(TM_IDX_SHFTD_DF_PER_TM)

TM_IDX_SHFTD_DF = pd.concat(TM_IDX_SHFTD_DF).sort_index(axis=0)
TM_IDX_SHFTD_DF

Team_Curr_Gm                             
                                      index               Team_id H/A   
index Team_id              H/A                                          
-1    -1                   -1            -1                    -1  -1  \
 0    /teams/ATL/2023.html  1             0  /teams/ATL/2023.html   1   
      /teams/BOS/2023.html  1             0  /teams/BOS/2023.html   1   
      /teams/BRK/2023.html  1             0  /teams/BRK/2023.html   1   
      /teams/CHI/2023.html  0             0  /teams/CHI/2023.html   0   
      /teams/CHO/2023.html  0             0  /teams/CHO/2023.html   0   
      /teams/CLE/2023.html  0             0  /teams/CLE/2023.html   0   
      /teams/DAL/2023.html  0             0  /teams/DAL/2023.html   0   
      /teams/DEN/2023.html  0             0  /teams/DEN/2023.html   0   
      /teams/DET/2023.html  1             0  /teams/DET/2023.html   1   
      /teams/GSW/2023.html  1             0  /teams/GSW/2023.html   1   
      /teams/HOU/2023.html  0             0  /teams/HOU/2023.html   0   
      /teams/IND/2023.html  1             0  /teams/IND/2023.html   1   
      /teams/LAC/2023.html  0             0  /teams/LAC/2023.html   0   
      /teams/LAL/2023.html  0             0  /teams/LAL/2023.html   0   
...                                     ...                   ...  ..   
 81   /teams/MIA/2023.html  1            81  /teams/MIA/2023.html   1   
      /teams/MIL/2023.html  0            81  /teams/MIL/2023.html   0   
      /teams/MIN/2023.html  1            81  /teams/MIN/2023.html   1   
      /teams/NOP/2023.html  0            81  /teams/NOP/2023.html   0   
      /teams/NYK/2023.html  1            81  /teams/NYK/2023.html   1   
      /teams/OKC/2023.html  1            81  /teams/OKC/2023.html   1   
      /teams/ORL/2023.html  0            81  /teams/ORL/2023.html   0   
      /teams/PHI/2023.html  0            81  /teams/PHI/2023.html   0   
      /teams/PHO/2023.html  1            81  /teams/PHO/2023.html   1   
      /teams/POR/2023.html  1            81  /teams/POR/2023.html   1   
      /teams/SAC/2023.html  0            81  /teams/SAC/2023.html   0   
      /teams/SAS/2023.html  0            81  /teams/SAS/2023.html   0   
      /teams/TOR/2023.html  1            81  /teams/TOR/2023.html   1   
      /teams/UTA/2023.html  0            81  /teams/UTA/2023.html   0   
      /teams/WAS/2023.html  1            81  /teams/WAS/2023.html   1   

                               Team_Prev_Gm_01                             
                                         index               Team_id H/A   
index Team_id              H/A                                             
-1    -1                   -1               -1                    -1  -1  \
 0    /teams/ATL/2023.html  1               -1                    -1  -1   
      /teams/BOS/2023.html  1               -1                    -1  -1   
      /teams/BRK/2023.html  1               -1                    -1  -1   
      /teams/CHI/2023.html  0               -1                    -1  -1   
      /teams/CHO/2023.html  0               -1                    -1  -1   
      /teams/CLE/2023.html  0               -1                    -1  -1   
      /teams/DAL/2023.html  0               -1                    -1  -1   
      /teams/DEN/2023.html  0               -1                    -1  -1   
      /teams/DET/2023.html  1               -1                    -1  -1   
      /teams/GSW/2023.html  1               -1                    -1  -1   
      /teams/HOU/2023.html  0               -1                    -1  -1   
      /teams/IND/2023.html  1               -1                    -1  -1   
      /teams/LAC/2023.html  0               -1                    -1  -1   
      /teams/LAL/2023.html  0               -1                    -1  -1   
...                                        ...                   ...  ..   
 81   /teams/MIA/2023.html  1               80  /teams/MIA/2023.html   0   
      /teams/MIL/2023.html  

In [147]:
OPP_IDX_PREV_DF = TM_IDX_SHFTD_DF.loc[pd.MultiIndex.from_frame(OPP_IDX_FACTS_DF)]
OPP_IDX_PREV_DF.index = TM_IDX_SHFTD_DF.index
ORG_COL_NAMES = OPP_IDX_PREV_DF.columns.levels[0]
NEW_COL_NAMES = [col_name.replace('Team','Opp') for col_name in ORG_COL_NAMES]
OPP_IDX_PREV_DF.rename(columns=dict(zip(ORG_COL_NAMES,NEW_COL_NAMES)),inplace=True)
OPP_IDX_PREV_DF

Opp_Curr_Gm                             
                                     index               Team_id H/A   
index Team_id              H/A                                         
-1    -1                   -1           -1                    -1  -1  \
 0    /teams/ATL/2023.html  1            0  /teams/HOU/2023.html   0   
      /teams/BOS/2023.html  1            0  /teams/PHI/2023.html   0   
      /teams/BRK/2023.html  1            0  /teams/NOP/2023.html   0   
      /teams/CHI/2023.html  0            0  /teams/MIA/2023.html   1   
      /teams/CHO/2023.html  0            0  /teams/SAS/2023.html   1   
      /teams/CLE/2023.html  0            0  /teams/TOR/2023.html   1   
      /teams/DAL/2023.html  0            0  /teams/PHO/2023.html   1   
      /teams/DEN/2023.html  0            0  /teams/UTA/2023.html   1   
      /teams/DET/2023.html  1            0  /teams/ORL/2023.html   0   
      /teams/GSW/2023.html  1            0  /teams/LAL/2023.html   0   
      /teams/HOU/2023.html  0            0  /teams/ATL/2023.html   1   
      /teams/IND/2023.html  1            0  /teams/WAS/2023.html   0   
      /teams/LAC/2023.html  0            1  /teams/LAL/2023.html   1   
      /teams/LAL/2023.html  0            0  /teams/GSW/2023.html   1   
...                                    ...                   ...  ..   
 81   /teams/MIA/2023.html  1           81  /teams/ORL/2023.html   0   
      /teams/MIL/2023.html  0           81  /teams/TOR/2023.html   1   
      /teams/MIN/2023.html  1           81  /teams/NOP/2023.html   0   
      /teams/NOP/2023.html  0           81  /teams/MIN/2023.html   1   
      /teams/NYK/2023.html  1           81  /teams/IND/2023.html   0   
      /teams/OKC/2023.html  1           81  /teams/MEM/2023.html   0   
      /teams/ORL/2023.html  0           81  /teams/MIA/2023.html   1   
      /teams/PHI/2023.html  0           81  /teams/BRK/2023.html   1   
      /teams/PHO/2023.html  1           81  /teams/LAC/2023.html   0   
      /teams/POR/2023.html  1           81  /teams/GSW/2023.html   0   
      /teams/SAC/2023.html  0           81  /teams/DEN/2023.html   1   
      /teams/SAS/2023.html  0           81  /teams/DAL/2023.html   1   
      /teams/TOR/2023.html  1           81  /teams/MIL/2023.html   0   
      /teams/UTA/2023.html  0           81  /teams/LAL/2023.html   1   
      /teams/WAS/2023.html  1           81  /teams/HOU/2023.html   0   

                               Opp_Prev_Gm_01                             
                                        index               Team_id H/A   
index Team_id              H/A                                            
-1    -1                   -1              -1                    -1  -1  \
 0    /teams/ATL/2023.html  1              -1                    -1  -1   
      /teams/BOS/2023.html  1              -1                    -1  -1   
      /teams/BRK/2023.html  1              -1                    -1  -1   
      /teams/CHI/2023.html  0              -1                    -1  -1   
      /teams/CHO/2023.html  0              -1                    -1  -1   
      /teams/CLE/2023.html  0              -1                    -1  -1   
      /teams/DAL/2023.html  0              -1                    -1  -1   
      /teams/DEN/2023.html  0              -1                    -1  -1   
      /teams/DET/2023.html  1              -1                    -1  -1   
      /teams/GSW/2023.html  1              -1                    -1  -1   
      /teams/HOU/2023.html  0              -1                    -1  -1   
      /teams/IND/2023.html  1              -1                    -1  -1   
      /teams/LAC/2023.html  0               0  /teams/LAL/2023.html   0   
      /teams/LAL/2023.html  0              -1                    -1  -1   
...                                       ...                   ...  ..   
 81   /teams/MIA/2023.html  1              80  /teams/ORL/2023.html   0   
      /teams/MIL/2023.html  0              80  /teams/TOR/2023.html   0   
      /

In [148]:
# Team previous 10 opponents
# Source: Team previous 10 games, keep the same index but flip the values Team -> Opp
# Replace each Team column with their corresponding Opp column:
TM_PREV_OPP_IDX_DF = []
TM_IDX_SHFTD_DF # Team previous 10 games
for col in TM_IDX_SHFTD_DF.columns.levels[0]:
    OPP_IDX = OPP_IDX_FACTS_DF.loc[pd.MultiIndex.from_frame(TM_IDX_SHFTD_DF[col])]
    OPP_IDX.index = TM_IDX_SHFTD_DF.index
    OPP_IDX.rename(columns={'Opp':col.replace('Gm','Gm_Opp')},inplace=True)
    TM_PREV_OPP_IDX_DF.append(OPP_IDX)

TM_PREV_OPP_IDX_DF = pd.concat(TM_PREV_OPP_IDX_DF,axis=1)
TM_PREV_OPP_IDX_DF

Team_Curr_Gm_Opp                             
                                          index               Team_id H/A   
index Team_id              H/A                                              
-1    -1                   -1                -1                    -1  -1  \
 0    /teams/ATL/2023.html  1                 0  /teams/HOU/2023.html   0   
      /teams/BOS/2023.html  1                 0  /teams/PHI/2023.html   0   
      /teams/BRK/2023.html  1                 0  /teams/NOP/2023.html   0   
      /teams/CHI/2023.html  0                 0  /teams/MIA/2023.html   1   
      /teams/CHO/2023.html  0                 0  /teams/SAS/2023.html   1   
      /teams/CLE/2023.html  0                 0  /teams/TOR/2023.html   1   
      /teams/DAL/2023.html  0                 0  /teams/PHO/2023.html   1   
      /teams/DEN/2023.html  0                 0  /teams/UTA/2023.html   1   
      /teams/DET/2023.html  1                 0  /teams/ORL/2023.html   0   
      /teams/GSW/2023.html  1                 0  /teams/LAL/2023.html   0   
      /teams/HOU/2023.html  0                 0  /teams/ATL/2023.html   1   
      /teams/IND/2023.html  1                 0  /teams/WAS/2023.html   0   
      /teams/LAC/2023.html  0                 1  /teams/LAL/2023.html   1   
      /teams/LAL/2023.html  0                 0  /teams/GSW/2023.html   1   
...                                         ...                   ...  ..   
 81   /teams/MIA/2023.html  1                81  /teams/ORL/2023.html   0   
      /teams/MIL/2023.html  0                81  /teams/TOR/2023.html   1   
      /teams/MIN/2023.html  1                81  /teams/NOP/2023.html   0   
      /teams/NOP/2023.html  0                81  /teams/MIN/2023.html   1   
      /teams/NYK/2023.html  1                81  /teams/IND/2023.html   0   
      /teams/OKC/2023.html  1                81  /teams/MEM/2023.html   0   
      /teams/ORL/2023.html  0                81  /teams/MIA/2023.html   1   
      /teams/PHI/2023.html  0                81  /teams/BRK/2023.html   1   
      /teams/PHO/2023.html  1                81  /teams/LAC/2023.html   0   
      /teams/POR/2023.html  1                81  /teams/GSW/2023.html   0   
      /teams/SAC/2023.html  0                81  /teams/DEN/2023.html   1   
      /teams/SAS/2023.html  0                81  /teams/DAL/2023.html   1   
      /teams/TOR/2023.html  1                81  /teams/MIL/2023.html   0   
      /teams/UTA/2023.html  0                81  /teams/LAL/2023.html   1   
      /teams/WAS/2023.html  1                81  /teams/HOU/2023.html   0   

                               Team_Prev_Gm_Opp_01                             
                                             index               Team_id H/A   
index Team_id              H/A                                                 
-1    -1                   -1                   -1                    -1  -1  \
 0    /teams/ATL/2023.html  1                   -1                    -1  -1   
      /teams/BOS/2023.html  1                   -1                    -1  -1   
      /teams/BRK/2023.html  1                   -1                    -1  -1   
      /teams/CHI/2023.html  0                   -1                    -1  -1   
      /teams/CHO/2023.html  0                   -1                    -1  -1   
      /teams/CLE/2023.html  0                   -1                    -1  -1   
      /teams/DAL/2023.html  0                   -1                    -1  -1   
      /teams/DEN/2023.html  0                   -1                    -1  -1   
      /teams/DET/2023.html  1                   -1                    -1  -1   
      /teams/GSW/2023.html  1                   -1                    -1  -1   
      /teams/HOU/2023.html  0                   -1                    -1  -1   
      /teams/IND/2023.html  1                   -1                    -1  -1   
      /teams/LAC/2023.html  0                   -1                    -1  -1   
      /teams/LAL/2023.html  0                   -1  

In [149]:
# Opp previous 10 opponents
# Indexed by Team, but values are Opp and Opp's previous 10 opponents
# Source: Previous 10 games as values, sort index by Opp

OPP_PREV_OPP_IDX_DF         = TM_PREV_OPP_IDX_DF.loc[pd.MultiIndex.from_frame(OPP_IDX_FACTS_DF)].copy()
OPP_PREV_OPP_IDX_DF.index   = TM_PREV_OPP_IDX_DF.index
SRC_COL_NAMES = TM_PREV_OPP_IDX_DF.columns.levels[0]
TGT_COL_NAMES = [col_name.replace('Team','Opp') for col_name in SRC_COL_NAMES]
OPP_PREV_OPP_IDX_DF.rename(columns=dict(zip(SRC_COL_NAMES,TGT_COL_NAMES)),inplace=True)
OPP_PREV_OPP_IDX_DF

Opp_Curr_Gm_Opp                             
                                         index               Team_id H/A   
index Team_id              H/A                                             
-1    -1                   -1               -1                    -1  -1  \
 0    /teams/ATL/2023.html  1                0  /teams/ATL/2023.html   1   
      /teams/BOS/2023.html  1                0  /teams/BOS/2023.html   1   
      /teams/BRK/2023.html  1                0  /teams/BRK/2023.html   1   
      /teams/CHI/2023.html  0                0  /teams/CHI/2023.html   0   
      /teams/CHO/2023.html  0                0  /teams/CHO/2023.html   0   
      /teams/CLE/2023.html  0                0  /teams/CLE/2023.html   0   
      /teams/DAL/2023.html  0                0  /teams/DAL/2023.html   0   
      /teams/DEN/2023.html  0                0  /teams/DEN/2023.html   0   
      /teams/DET/2023.html  1                0  /teams/DET/2023.html   1   
      /teams/GSW/2023.html  1                0  /teams/GSW/2023.html   1   
      /teams/HOU/2023.html  0                0  /teams/HOU/2023.html   0   
      /teams/IND/2023.html  1                0  /teams/IND/2023.html   1   
      /teams/LAC/2023.html  0                0  /teams/LAC/2023.html   0   
      /teams/LAL/2023.html  0                0  /teams/LAL/2023.html   0   
...                                        ...                   ...  ..   
 81   /teams/MIA/2023.html  1               81  /teams/MIA/2023.html   1   
      /teams/MIL/2023.html  0               81  /teams/MIL/2023.html   0   
      /teams/MIN/2023.html  1               81  /teams/MIN/2023.html   1   
      /teams/NOP/2023.html  0               81  /teams/NOP/2023.html   0   
      /teams/NYK/2023.html  1               81  /teams/NYK/2023.html   1   
      /teams/OKC/2023.html  1               81  /teams/OKC/2023.html   1   
      /teams/ORL/2023.html  0               81  /teams/ORL/2023.html   0   
      /teams/PHI/2023.html  0               81  /teams/PHI/2023.html   0   
      /teams/PHO/2023.html  1               81  /teams/PHO/2023.html   1   
      /teams/POR/2023.html  1               81  /teams/POR/2023.html   1   
      /teams/SAC/2023.html  0               81  /teams/SAC/2023.html   0   
      /teams/SAS/2023.html  0               81  /teams/SAS/2023.html   0   
      /teams/TOR/2023.html  1               81  /teams/TOR/2023.html   1   
      /teams/UTA/2023.html  0               81  /teams/UTA/2023.html   0   
      /teams/WAS/2023.html  1               81  /teams/WAS/2023.html   1   

                               Opp_Prev_Gm_Opp_01                             
                                            index               Team_id H/A   
index Team_id              H/A                                                
-1    -1                   -1                  -1                    -1  -1  \
 0    /teams/ATL/2023.html  1                  -1                    -1  -1   
      /teams/BOS/2023.html  1                  -1                    -1  -1   
      /teams/BRK/2023.html  1                  -1                    -1  -1   
      /teams/CHI/2023.html  0                  -1                    -1  -1   
      /teams/CHO/2023.html  0                  -1                    -1  -1   
      /teams/CLE/2023.html  0                  -1                    -1  -1   
      /teams/DAL/2023.html  0                  -1                    -1  -1   
      /teams/DEN/2023.html  0                  -1                    -1  -1   
      /teams/DET/2023.html  1                  -1                    -1  -1   
      /teams/GSW/2023.html  1                  -1                    -1  -1   
      /teams/HOU/2023.html  0                  -1                    -1  -1   
      /teams/IND/2023.html  1                  -1                    -1  -1   
      /teams/LAC/2023.html  0                   0  /teams/GSW/2023.html   1   
      /teams/LAL/2023.html  0                  -1                    -1  -1   
...                     

In [164]:
# TM = '/teams/BOS/2023.html'
# VEN = 0
TM_IDX_VEN_PREV_GM_DF_LIST_DF = []
for TM in set(pd.unique(TM_IDX_FACTS_DF[('Team','Team_id')])).difference([-1]):
    for VEN in [0,1]:
        TM_IDX_VEN_PREV_GM_DF = TM_IDX_FACTS_DF.loc[(slice(None),TM,VEN),:]
        TM_IDX_VEN_PREV_GM_DF_LIST = [TM_IDX_VEN_PREV_GM_DF.rename(columns={'Team':f'Team_Curr_Gm'})]
        for i in range(1,len(TM_IDX_VEN_PREV_GM_DF)):
            TM_IDX_VEN_PREV_GM_DF_I = TM_IDX_VEN_PREV_GM_DF.copy().shift(i,fill_value=-1) 
            Gm_No = f'0{i}' if i < 10 else f'{i}'
            TM_IDX_VEN_PREV_GM_DF_I.rename(columns={'Team':f'Team_Ven_Prev_Gm_{Gm_No}'},inplace=True)
            TM_IDX_VEN_PREV_GM_DF_LIST.append(TM_IDX_VEN_PREV_GM_DF_I)
        TM_IDX_VEN_PREV_GM_DF = pd.concat(TM_IDX_VEN_PREV_GM_DF_LIST,axis=1)
        TM_IDX_VEN_PREV_GM_DF_LIST_DF.append(TM_IDX_VEN_PREV_GM_DF)

TM_IDX_VEN_PREV_GM_DF_LIST_DF = pd.concat(TM_IDX_VEN_PREV_GM_DF_LIST_DF).sort_index(axis=0)
TM_IDX_VEN_PREV_GM_DF_LIST_DF

Team_Curr_Gm                             
                                      index               Team_id H/A   
index Team_id              H/A                                          
0     /teams/ATL/2023.html 1              0  /teams/ATL/2023.html   1  \
      /teams/BOS/2023.html 1              0  /teams/BOS/2023.html   1   
      /teams/BRK/2023.html 1              0  /teams/BRK/2023.html   1   
      /teams/CHI/2023.html 0              0  /teams/CHI/2023.html   0   
      /teams/CHO/2023.html 0              0  /teams/CHO/2023.html   0   
      /teams/CLE/2023.html 0              0  /teams/CLE/2023.html   0   
      /teams/DAL/2023.html 0              0  /teams/DAL/2023.html   0   
      /teams/DEN/2023.html 0              0  /teams/DEN/2023.html   0   
      /teams/DET/2023.html 1              0  /teams/DET/2023.html   1   
      /teams/GSW/2023.html 1              0  /teams/GSW/2023.html   1   
      /teams/HOU/2023.html 0              0  /teams/HOU/2023.html   0   
      /teams/IND/2023.html 1              0  /teams/IND/2023.html   1   
      /teams/LAC/2023.html 0              0  /teams/LAC/2023.html   0   
      /teams/LAL/2023.html 0              0  /teams/LAL/2023.html   0   
      /teams/MEM/2023.html 1              0  /teams/MEM/2023.html   1   
...                                     ...                   ...  ..   
81    /teams/MIA/2023.html 1             81  /teams/MIA/2023.html   1   
      /teams/MIL/2023.html 0             81  /teams/MIL/2023.html   0   
      /teams/MIN/2023.html 1             81  /teams/MIN/2023.html   1   
      /teams/NOP/2023.html 0             81  /teams/NOP/2023.html   0   
      /teams/NYK/2023.html 1             81  /teams/NYK/2023.html   1   
      /teams/OKC/2023.html 1             81  /teams/OKC/2023.html   1   
      /teams/ORL/2023.html 0             81  /teams/ORL/2023.html   0   
      /teams/PHI/2023.html 0             81  /teams/PHI/2023.html   0   
      /teams/PHO/2023.html 1             81  /teams/PHO/2023.html   1   
      /teams/POR/2023.html 1             81  /teams/POR/2023.html   1   
      /teams/SAC/2023.html 0             81  /teams/SAC/2023.html   0   
      /teams/SAS/2023.html 0             81  /teams/SAS/2023.html   0   
      /teams/TOR/2023.html 1             81  /teams/TOR/2023.html   1   
      /teams/UTA/2023.html 0             81  /teams/UTA/2023.html   0   
      /teams/WAS/2023.html 1             81  /teams/WAS/2023.html   1   

                               Team_Ven_Prev_Gm_01                             
                                             index               Team_id H/A   
index Team_id              H/A                                                 
0     /teams/ATL/2023.html 1                    -1                    -1  -1  \
      /teams/BOS/2023.html 1                    -1                    -1  -1   
      /teams/BRK/2023.html 1                    -1                    -1  -1   
      /teams/CHI/2023.html 0                    -1                    -1  -1   
      /teams/CHO/2023.html 0                    -1                    -1  -1   
      /teams/CLE/2023.html 0                    -1                    -1  -1   
      /teams/DAL/2023.html 0                    -1                    -1  -1   
      /teams/DEN/2023.html 0                    -1                    -1  -1   
      /teams/DET/2023.html 1                    -1                    -1  -1   
      /teams/GSW/2023.html 1                    -1                    -1  -1   
      /teams/HOU/2023.html 0                    -1                    -1  -1   
      /teams/IND/2023.html 1                    -1                    -1  -1   
      /teams/LAC/2023.html 0                    -1                    -1  -1   
      /teams/LAL/2023.html 0                    -1                    -1  -1   
      /teams/MEM/2023.html 1                    -1                    -1  -1   
...                                            ...                   ...  ..   
81    /teams/MIA/2023.html 1

In [276]:

TEAMS = pd.unique(TM_OPP_IDX_FACTS_DF[('Team','Team_id')].drop(-1))
TM_IDX_H2H_FACTS_SHIFTED_DF_LIST = []
for TM_ID in TEAMS:
    for OPP_ID in TEAMS:
        MATCH_UP_FILTER = (TM_OPP_IDX_FACTS_DF[('Team','Team_id')] == TM_ID) & (TM_OPP_IDX_FACTS_DF[('Opp','Team_id')] == OPP_ID)
        TM_OPP_IDX_H2H_FACTS  =  TM_OPP_IDX_FACTS_DF[MATCH_UP_FILTER].sort_index()
        if len(TM_OPP_IDX_H2H_FACTS) == 0:
            continue
        TM_IDX_H2H_FACTS_SHIFTED_LIST  = [TM_OPP_IDX_H2H_FACTS.rename(columns={'Team':'Team_Curr_H2H_Gm','Opp':'Opp_Curr_H2H_Gm'})]
        for i in range(1,len(TM_OPP_IDX_H2H_FACTS)):
            TM_IDX_H2H_FACTS_SHIFTED = TM_OPP_IDX_H2H_FACTS.shift(i,fill_value=-1)
            Gm_No = f'0{i}' if i < 10 else f'{i}'
            TM_IDX_H2H_FACTS_SHIFTED.rename(columns={'Team':f'Team_Prev_H2H_Gm_{Gm_No}','Opp':f'Opp_Prev_H2H_Gm_{Gm_No}'},inplace=True)
            TM_IDX_H2H_FACTS_SHIFTED_LIST.append(TM_IDX_H2H_FACTS_SHIFTED)

        TM_IDX_H2H_FACTS_SHIFTED_DF = pd.concat(TM_IDX_H2H_FACTS_SHIFTED_LIST,axis=1).fillna(-1)
        TM_IDX_H2H_FACTS_SHIFTED_DF_LIST.append(TM_IDX_H2H_FACTS_SHIFTED_DF)

TM_IDX_H2H_FACTS_SHIFTED_DF_LIST = pd.concat(TM_IDX_H2H_FACTS_SHIFTED_DF_LIST).fillna(-1).sort_index(axis=0)
for i,col in enumerate(TM_IDX_H2H_FACTS_SHIFTED_DF_LIST.columns):
    if col[1] in ['index','H/A']:
        INT_COL = TM_IDX_H2H_FACTS_SHIFTED_DF_LIST.pop(col).astype(int)
        TM_IDX_H2H_FACTS_SHIFTED_DF_LIST.insert(i,col,INT_COL)

TM_IDX_H2H_FACTS_SHIFTED_DF_LIST

Team_Curr_H2H_Gm                             
                                          index               Team_id H/A   
index Team_id              H/A                                              
0     /teams/ATL/2023.html 1                  0  /teams/ATL/2023.html   1  \
      /teams/BOS/2023.html 1                  0  /teams/BOS/2023.html   1   
      /teams/BRK/2023.html 1                  0  /teams/BRK/2023.html   1   
      /teams/CHI/2023.html 0                  0  /teams/CHI/2023.html   0   
      /teams/CHO/2023.html 0                  0  /teams/CHO/2023.html   0   
      /teams/CLE/2023.html 0                  0  /teams/CLE/2023.html   0   
      /teams/DAL/2023.html 0                  0  /teams/DAL/2023.html   0   
      /teams/DEN/2023.html 0                  0  /teams/DEN/2023.html   0   
      /teams/DET/2023.html 1                  0  /teams/DET/2023.html   1   
      /teams/GSW/2023.html 1                  0  /teams/GSW/2023.html   1   
      /teams/HOU/2023.html 0                  0  /teams/HOU/2023.html   0   
      /teams/IND/2023.html 1                  0  /teams/IND/2023.html   1   
      /teams/LAC/2023.html 0                  0  /teams/LAC/2023.html   0   
      /teams/LAL/2023.html 0                  0  /teams/LAL/2023.html   0   
      /teams/MEM/2023.html 1                  0  /teams/MEM/2023.html   1   
...                                         ...                   ...  ..   
81    /teams/MIA/2023.html 1                 81  /teams/MIA/2023.html   1   
      /teams/MIL/2023.html 0                 81  /teams/MIL/2023.html   0   
      /teams/MIN/2023.html 1                 81  /teams/MIN/2023.html   1   
      /teams/NOP/2023.html 0                 81  /teams/NOP/2023.html   0   
      /teams/NYK/2023.html 1                 81  /teams/NYK/2023.html   1   
      /teams/OKC/2023.html 1                 81  /teams/OKC/2023.html   1   
      /teams/ORL/2023.html 0                 81  /teams/ORL/2023.html   0   
      /teams/PHI/2023.html 0                 81  /teams/PHI/2023.html   0   
      /teams/PHO/2023.html 1                 81  /teams/PHO/2023.html   1   
      /teams/POR/2023.html 1                 81  /teams/POR/2023.html   1   
      /teams/SAC/2023.html 0                 81  /teams/SAC/2023.html   0   
      /teams/SAS/2023.html 0                 81  /teams/SAS/2023.html   0   
      /teams/TOR/2023.html 1                 81  /teams/TOR/2023.html   1   
      /teams/UTA/2023.html 0                 81  /teams/UTA/2023.html   0   
      /teams/WAS/2023.html 1                 81  /teams/WAS/2023.html   1   

                               Opp_Curr_H2H_Gm                             
                                         index               Team_id H/A   
index Team_id              H/A                                             
0     /teams/ATL/2023.html 1                 0  /teams/HOU/2023.html   0  \
      /teams/BOS/2023.html 1                 0  /teams/PHI/2023.html   0   
      /teams/BRK/2023.html 1                 0  /teams/NOP/2023.html   0   
      /teams/CHI/2023.html 0                 0  /teams/MIA/2023.html   1   
      /teams/CHO/2023.html 0                 0  /teams/SAS/2023.html   1   
      /teams/CLE/2023.html 0                 0  /teams/TOR/2023.html   1   
      /teams/DAL/2023.html 0                 0  /teams/PHO/2023.html   1   
      /teams/DEN/2023.html 0                 0  /teams/UTA/2023.html   1   
      /teams/DET/2023.html 1                 0  /teams/ORL/2023.html   0   
      /teams/GSW/2023.html 1                 0  /teams/LAL/2023.html   0   
      /teams/HOU/2023.html 0                 0  /teams/ATL/2023.html   1   
      /teams/IND/2023.html 1                 0  /teams/WAS/2023.html   0   
      /teams/LAC/2023.html 0                 1  /teams/LAL/2023.html   1   
      /teams/LAL/2023.html 0                 0  /teams/GSW/2023.html   1   
      /teams/MEM/2023.html 1                 0  /teams/NYK/2023.html   0   
...                                        .

In [274]:
TM_IDX_H2H_FACTS_SHIFTED_DF_LIST.to_csv('test.csv')
df = pd.read_csv('test.csv',header=[0,1],index_col=[0,1,2])
df

Team_Curr_H2H_Gm                             
                                          index               Team_id H/A   
index Team_id              H/A                                              
14    /teams/ATL/2023.html 1                 14  /teams/ATL/2023.html   1  \
67    /teams/ATL/2023.html 1                 67  /teams/ATL/2023.html   1   
81    /teams/ATL/2023.html 0                 81  /teams/ATL/2023.html   0   
25    /teams/ATL/2023.html 0                 25  /teams/ATL/2023.html   0   
34    /teams/ATL/2023.html 1                 34  /teams/ATL/2023.html   1   
60    /teams/ATL/2023.html 1                 60  /teams/ATL/2023.html   1   
76    /teams/ATL/2023.html 0                 76  /teams/ATL/2023.html   0   
26    /teams/ATL/2023.html 1                 26  /teams/ATL/2023.html   1   
31    /teams/ATL/2023.html 1                 31  /teams/ATL/2023.html   1   
47    /teams/ATL/2023.html 0                 47  /teams/ATL/2023.html   0   
78    /teams/ATL/2023.html 0                 78  /teams/ATL/2023.html   0   
2     /teams/ATL/2023.html 1                  2  /teams/ATL/2023.html   1   
29    /teams/ATL/2023.html 0                 29  /teams/ATL/2023.html   0   
46    /teams/ATL/2023.html 1                 46  /teams/ATL/2023.html   1   
57    /teams/ATL/2023.html 0                 57  /teams/ATL/2023.html   0   
...                                         ...                   ...  ..   
34    /teams/WAS/2023.html 1                 34  /teams/WAS/2023.html   1   
67    /teams/WAS/2023.html 0                 67  /teams/WAS/2023.html   0   
31    /teams/WAS/2023.html 0                 31  /teams/WAS/2023.html   0   
35    /teams/WAS/2023.html 1                 35  /teams/WAS/2023.html   1   
50    /teams/WAS/2023.html 1                 50  /teams/WAS/2023.html   1   
56    /teams/WAS/2023.html 0                 56  /teams/WAS/2023.html   0   
33    /teams/WAS/2023.html 0                 33  /teams/WAS/2023.html   0   
70    /teams/WAS/2023.html 1                 70  /teams/WAS/2023.html   1   
49    /teams/WAS/2023.html 0                 49  /teams/WAS/2023.html   0   
73    /teams/WAS/2023.html 1                 73  /teams/WAS/2023.html   1   
61    /teams/WAS/2023.html 1                 61  /teams/WAS/2023.html   1   
62    /teams/WAS/2023.html 1                 62  /teams/WAS/2023.html   1   
74    /teams/WAS/2023.html 0                 74  /teams/WAS/2023.html   0   
12    /teams/WAS/2023.html 1                 12  /teams/WAS/2023.html   1   
32    /teams/WAS/2023.html 0                 32  /teams/WAS/2023.html   0   

                               Opp_Curr_H2H_Gm                             
                                         index               Team_id H/A   
index Team_id              H/A                                             
14    /teams/ATL/2023.html 1                14  /teams/BOS/2023.html   0  \
67    /teams/ATL/2023.html 1                67  /teams/BOS/2023.html   0   
81    /teams/ATL/2023.html 0                81  /teams/BOS/2023.html   1   
25    /teams/ATL/2023.html 0                26  /teams/BRK/2023.html   1   
34    /teams/ATL/2023.html 1                34  /teams/BRK/2023.html   0   
60    /teams/ATL/2023.html 1                59  /teams/BRK/2023.html   0   
76    /teams/ATL/2023.html 0                76  /teams/BRK/2023.html   1   
26    /teams/ATL/2023.html 1                25  /teams/CHI/2023.html   0   
31    /teams/ATL/2023.html 1                30  /teams/CHI/2023.html   0   
47    /teams/ATL/2023.html 0                45  /teams/CHI/2023.html   1   
78    /teams/ATL/2023.html 0                78  /teams/CHI/2023.html   1   
2     /teams/ATL/2023.html 1                 2  /teams/CHO/2023.html   0   
29    /teams/ATL/2023.html 0                28  /teams/CHO/2023.html   1   
46    /teams/ATL/2023.html 1                46  /teams/CHO/2023.html   0   
57    /teams/ATL/2023.html 0                58  /teams/CHO/2023.html   1   
...                                        .

In [275]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2460 entries, (14, '/teams/ATL/2023.html', 1) to (32, '/teams/WAS/2023.html', 0)
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   (Team_Curr_H2H_Gm, index)       2460 non-null   int64 
 1   (Team_Curr_H2H_Gm, Team_id)     2460 non-null   object
 2   (Team_Curr_H2H_Gm, H/A)         2460 non-null   int64 
 3   (Opp_Curr_H2H_Gm, index)        2460 non-null   int64 
 4   (Opp_Curr_H2H_Gm, Team_id)      2460 non-null   object
 5   (Opp_Curr_H2H_Gm, H/A)          2460 non-null   int64 
 6   (Team_Prev_H2H_Gm_01, index)    2460 non-null   int64 
 7   (Team_Prev_H2H_Gm_01, Team_id)  2460 non-null   object
 8   (Team_Prev_H2H_Gm_01, H/A)      2460 non-null   int64 
 9   (Opp_Prev_H2H_Gm_01, index)     2460 non-null   int64 
 10  (Opp_Prev_H2H_Gm_01, Team_id)   2460 non-null   object
 11  (Opp_Prev_H2H_Gm_01, H/A)       2460 non-null   int

In [ ]:
TM_IDX_PRIOR_GM_DF = TM_IDX_FACTS_DF.copy().drop(columns=[('Team','H/A'),('Opp','H/A')])
# TGL_BASIC_INFO_ADJ_DF.set_index(pd.MultiIndex.from_frame(TGL_BASIC_INFO_ADJ_DF['Team']),inplace=True,drop=False)
TM_IDX_PRIOR_GM_DF.loc[:,('Team','index')] = TM_IDX_PRIOR_GM_DF['Team']['index'] - 1
TM_IDX_PRIOR_GM_DF.loc[:,('Opp','index')] = TM_IDX_PRIOR_GM_DF['Opp']['index'] - 1
TM_IDX_PRIOR_GM_DF.rename(columns={'Team':'Team_Prior_Gm','Opp':'Opp_Prior_Gm'},inplace=True)
TM_IDX_PRIOR_GM_DF.sort_index(axis=0,inplace=True)
TM_IDX_PRIOR_GM_DF

In [ ]:
org_team_index = (20,'/teams/GSW/2023.html')
TM_IDX_PRIOR_GM_DF.loc[org_team_index]
prior_team_index = tuple(TM_IDX_PRIOR_GM_DF.loc[org_team_index,'Team_Prior_Gm'])
prior_opp_index = tuple(TM_IDX_PRIOR_GM_DF.loc[org_team_index,'Opp_Prior_Gm'])
GLAN_SS_PER_RD_DF.loc[[prior_team_index,prior_opp_index]]

In [ ]:
# Get gamelogs
gamelog_basic = './data-parsed/teams/BOS/2023/tgl_basic.csv'
# Get computed gamelogs
info_days_btw_gms = './data-computed/teams/BOS/2023/tgl_basic/info_days_btw_gms.csv'
info_gm_results = './data-computed/teams/BOS/2023/tgl_basic/info_gm_results.csv'
# Get agg_norm gamelogs



In [ ]:
index = 22
team_id = '/teams/GSW/2023.html'
ha = 1

MATCH_UP_FILTER = (TM_IDX_FACTS_DF[('Team','Team_id')] == '/teams/GSW/2023.html') & (TM_IDX_FACTS_DF[('Team','H/A')] == 1)
TM_IDX_VEN_FACTS =  TM_IDX_FACTS_DF[MATCH_UP_FILTER].sort_index()
TM_IDX_VEN_FACTS

In [ ]:
TM_IDX_VEN_PRIOR_DF = TM_IDX_VEN_FACTS['Team'].copy()
# TM_IDX_VEN_PRIOR_DF.set_index(['index','Team_id'],inplace=True,drop=False)
TM_IDX_VEN_PRIOR_DF['index'] = TM_IDX_VEN_PRIOR_DF['index'].shift(1,fill_value=-1)
TM_IDX_VEN_PRIOR_DF

In [ ]:
df_list = []
for team_id in LG_SS_HTML_DICT[LG_SS_HTML]:
    for ha in [0,1]:
        MATCH_UP_FILTER = (TM_IDX_FACTS_DF[('Team','Team_id')] == team_id) & (TM_IDX_FACTS_DF[('Team','H/A')] == ha)
        TM_IDX_VEN_FACTS    =  TM_IDX_FACTS_DF[MATCH_UP_FILTER].sort_index()
        TM_IDX_VEN_PRIOR_DF = TM_IDX_VEN_FACTS['Team'].copy()
        TM_IDX_VEN_PRIOR_DF['index'] = TM_IDX_VEN_PRIOR_DF['index'].shift(1,fill_value=-1)
        df_list.append(TM_IDX_VEN_PRIOR_DF)
TM_IDX_VEN_PRIOR_DF = pd.concat(df_list)
# TM_IDX_VEN_PRIOR_DF.sort_index(inplace=True)
TM_IDX_VEN_PRIOR_DF

In [ ]:
OPP_FACTS = TM_IDX_FACTS_DF.loc[TM_IDX_VEN_PRIOR_DF.index,'Opp'][['index','Team_id']]
OPP_IDX_VEN_PRIOR_DF = pd.DataFrame(data = TM_IDX_VEN_PRIOR_DF.loc[pd.MultiIndex.from_frame(OPP_FACTS),:].values,
                                index = TM_IDX_VEN_PRIOR_DF.index, columns=TM_IDX_VEN_PRIOR_DF.columns)
# OPP_IDX_VEN_PRIOR_DF
TM_OPP_IDX_VEN_PRIOR_DF = pd.concat([TM_IDX_VEN_PRIOR_DF,OPP_IDX_VEN_PRIOR_DF],axis=1,keys=['Team_Ven_Prior','Opp_Ven_Prior'])
TM_OPP_IDX_VEN_PRIOR_DF

TM_IDX_FACTS_DF

In [ ]:
TM_IDX_FACTS_DF
TM_ID = '/teams/GSW/2023.html'
OPP_ID = '/teams/PHI/2023.html'

MATCH_UP_FILTER = (TM_IDX_FACTS_DF[('Team','Team_id')] == TM_ID) & (TM_IDX_FACTS_DF[('Opp','Team_id')] == OPP_ID)
TM_IDX_H2H_FACTS    =  TM_IDX_FACTS_DF[MATCH_UP_FILTER].sort_index()
TM_IDX_H2H_FACTS

In [ ]:
TM_IDX_H2H_FACTS_SHIFTED_LIST  = []
for i in range(len(TM_IDX_H2H_FACTS)):
    TM_IDX_H2H_FACTS_SHIFTED = TM_IDX_H2H_FACTS.shift(i)
    TM_IDX_H2H_FACTS_SHIFTED_LIST.append(TM_IDX_H2H_FACTS_SHIFTED)
    # print(TM_IDX_H2H_FACTS.iloc[i]['Team']['index'],TM_IDX_H2H_FACTS.iloc[i]['Opp']['index'])

TM_IDX_H2H_FACTS_SHIFTED_DF = pd.concat(TM_IDX_H2H_FACTS_SHIFTED_LIST,axis=1,
                                        keys=['Current_Gm'] + [f'Prev_H2H_Gm_{i}' for i in range(1,len(TM_IDX_H2H_FACTS_SHIFTED_LIST))])
TM_IDX_H2H_FACTS_SHIFTED_DF

In [ ]:
LG_SS_HTML_DICT_STR = load_file(f'{SRC_DIR}/league_seasons_html.txt')
LG_SS_HTML_DICT = ast.literal_eval(LG_SS_HTML_DICT_STR)

# For every league_season, get all team_seasons belonging to the league
for LG_SS_HTML in LG_SS_HTML_DICT.keys():
    LG_SS_DIR = parse_league_id(LG_SS_HTML)['body']
    BS_TM_INDEX_DF_LIST = []
    
    for TM_SS_HTML in LG_SS_HTML_DICT[LG_SS_HTML]:
        TM_SS_DIR = parse_team_season_id(TM_SS_HTML)['body'] # e.g: teams/BOS/2023
        TGL_BASIC_DF = pd.read_csv(f'{SRC_DIR}/{TM_SS_DIR}/tgl_basic.csv',header=[0,1]).rename(columns={'Unnamed: 0_level_0':'Match','Unnamed: 0_level_1':'index'})['Match']
        TGL_BASIC_DF = TGL_BASIC_DF.set_index('Boxscores_html_id').rename(columns={'Tm_html_id':'Team_id'})[['index','Team_id']]
        BS_TM_INDEX_DF_LIST.append(TGL_BASIC_DF)
    BS_TM_INDEX_DF = pd.concat(BS_TM_INDEX_DF_LIST).sort_index(axis=0)
    
    # Go through each match boxscores and add the corresponding opponent's index
    BS_TM_OPP_INDEX_DF_LIST = []
    for BS_INDEX in sorted(pd.unique(BS_TM_INDEX_DF.index)):
        BS_TM_DF = BS_TM_INDEX_DF.loc[BS_INDEX].copy()
        BS_TM_OPP_INDEX_DF_LIST.append(pd.concat([BS_TM_DF,BS_TM_DF.iloc[::-1]],axis=1,keys=['Team','Opp']))
    BS_TM_OPP_INDEX_DF = pd.concat(BS_TM_OPP_INDEX_DF_LIST)

    # Create an adjusted index table to point to the previous match index of the team and opponent
    BS_TM_OPP_INDEX_ADJ_DF = BS_TM_OPP_INDEX_DF.copy()
    BS_TM_OPP_INDEX_ADJ_DF.loc[:,('Team','index')] = BS_TM_OPP_INDEX_DF['Team']['index'] - 1
    BS_TM_OPP_INDEX_ADJ_DF.loc[:,('Opp','index')] = BS_TM_OPP_INDEX_DF['Opp']['index'] - 1
    BS_TM_OPP_INDEX_ADJ_DF.rename(columns={'Team':'Team_prior','Opp':'Opp_prior'},inplace=True)
    BS_TM_OPP_INDEX_ADJ_DF.set_index(pd.MultiIndex.from_frame(BS_TM_OPP_INDEX_DF['Team']),inplace=True)
        
